In [1]:
!pip install openai

  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached pydantic-2.6.4-py3-none-any.whl.metadata (85 kB)
  Using cached tqdm-4.66.2-py3-none-any.whl.metadata (57 kB)
  Using cached annotated_types-0.6.0-py3-none-any.whl.metadata (12 kB)
  Using cached pydantic_core-2.16.3-cp311-none-win_amd64.whl.metadata (6.6 kB)
   ---------------------------------------- 0.0/262.4 kB ? eta -:--:--
   ---------------------------------------- 0.0/262.4 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/262.4 kB 660.6 kB/s eta 0:00:01
   ---- ---------------------------------- 30.7/262.4 kB 660.6 kB/s eta 0:00:01
   ---- ---------------------------------- 30.7/262.4 kB 660.6 kB/s eta 0:00:01
   ------ -------------------------------- 41.0/262.4 kB 178.6 kB/s eta 0:00:02
   --------- ----------------------------- 61.4/262.4 kB 252.2 kB/s eta 0:00:01
   ------------- ------------------------- 92.2/262.4 kB 327.7 kB/s eta 0:00:01
   ----------------- -------------------

In [2]:
import openai
openai.api_key = "your-key"

In [3]:
prompt = "produce python code that produces personalized retention incentives based on churn predictions model in https://github.com/sunone5/telco_churn/blob/main/models/model.joblib"
# completion = openai.Completion.create(engine="text-davinci-003", prompt=prompt, max_tokens=1000) - Depricated > openai.Completion.create
completion = openai.completions.create(
  model="gpt-3.5-turbo-instruct",
  prompt=prompt,
  max_tokens=1000,
  temperature=0
)
print(completion.choices[0]['text'])

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}